#### pd.where #Titanic: masking and mapping
___ 
In my previous [Medium][1] write-up, I attempted, with success, using [***np.where***][2] to impute missing NaN/null value. I showed that the trick is the *nested np.where*. It was an attempt proffering alternative to the *if ... condition* often used in the [**#Titanic** dataset][3] to impute average values; typically for age

This follow-up, takes the *np.where* to the *native* pandas [***pd.where()***][4]. 
Unlike np.where which returns a given 'value' when condition is true and another when condition is false,  
pd.where on the otherhand returns the dataframe or column when condition is true and given value when the condition is false.  

The basic construct is `pd.where(condition, y)` | pd is actually pandas.DataFrame  

Before proceeding, let's take a look at the null/NaN value.
NaN (Not a Number) was formalised in the [IEEE Standard 754][5], published in 1985 and revised in 2008 and 2019. It is a standard for binary and decimal floating-point arithmetic. [NaN][6] as the name imply, cannot be engaged as number. In essence, attempts to use 'operators' (such as like (==), equal to (= or /eq)) will result in errors or unpredictable outcomes.

[Outcome]
For the type of *if ... condition* we are looking at, where we return a specific value when the condition is true or false,  
in my limited knowledge, the conclusion is that pd.where would not work, under the circumstances.

There are alternatives though. I'll take a look at some: [loc][7], [mask][8], and [map][9].

**[loc[]]**
The Pandas (DataFrame) [.loc][7] access a group of rows and columns by label(s). Due to its ability to handle boolean array and set values,   
we can leverage .loc to 'lookup' a condition and set value for a given column.
In the simplest form, this will be: `df.loc[(cond), data col to change] = value to change`

**[mask]**
The Pandas (DataFrame) [mask() function][8] works similarly to the np.where: replace a value when the condition is *true*.  
We can apply in similar manner to the Python's map() function.
NB: Pandas .where() on the other hand, returns self (object of same shape) when condition is *true*  
To get .mask() workig, I need to set [inplace flag to True][12]


**[map()]**
Python has a [built-in map() function][9]. It returns an iterator based on a function applied to (any number of) iterables.  
In our case, we can check our pclass column (col[1]) and substitute the values in our column of interest, the age (col[0]) with 37 and 39.  
To implement this substitution, we apply using [fillna()][10].  
In the simplest form: `df.col[1].map({1: 37, 2: 39}).fillna(24)`
Essentially, where age in NaN and pclass is '1' or '2', we substitute with '37' and '39' respectively,  
else, where age is NaN, we substitute with '24'


NB: I used a dynamically generated data as a proxy for the #titanic dataset.

[1]: <https://medium.com/@AKayode_research/np-where-the-titanic-83bdc0a16b11> "np.where"
[2]: <https://numpy.org/doc/stable/reference/generated/numpy.where.html> "NumPy. np.where (v1.23)"
[3]: <https://kaggle.com/competitions/titanic/data> "Kaggle Titanic dataset"
[4]: <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.where.html> "Pandas.where 1.5.2"
[5]: <https://standards.ieee.org/ieee/754/6210/> "IEEE NaN"
[6]: <https://python-course.eu/numerical-programming/dealing-with-nan.php> "NaN operator"
[7]: <https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html> "Pandas loc"
[8]: <https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mask.html> "Pandas Mask"
[9]: <https://docs.python.org/3/library/functions.html#map> "Python built-in map()"
[10]: <https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html> "Pandas fillna()"
[11]: <https://stackoverflow.com/a/74630735/20107918> "SO: pd map"
[12]: <https://stackoverflow.com/questions/61177777/possible-to-use-mask-or-where-in-pandas-on-a-whole-dataframe-but-change-only> "Pandas Mask inplace"


In [1]:
## import libraries
import pandas as pd
import numpy as np

In [2]:
## create data (in lieu of the Titanic dataset)
A = np.arange(20, 30, 3) #This will create a list with 4 elements (needed to ensure at least one NaN)

B1 = np.arange(3,6)
#Generate a 5 x 1 array* of ints between 1 and 3, inclusive:
B_add = np.random.default_rng().integers(1,4, size=(5))
B = np.append(B1, B_add) #concatenate or vstack for scalar
print(f'A: {A} | B: {B}')

## create dataframe, transposed, and check the created data
cols_test = pd.DataFrame([A,B]).T
cols_test[1] = cols_test[1].astype(int) #Ensure the pclass column is int
cols_test

A: [20 23 26 29] | B: [3 4 5 2 1 2 3 1]


,0,1
0,20.0,3
1,23.0,4
2,26.0,5
3,29.0,2
4,NaN,1
5,NaN,2
6,NaN,3
7,NaN,1


In [3]:
## stackoverflow
## https://stackoverflow.com/questions/56791544/using-pandas-df-where-on-multiple-columns-produces-unexpected-nan-values
## https://stackoverflow.com/questions/70170218/use-nested-if-condition-in-dataframe
## 

In [4]:
## I generate this data only to test (ensuring all options are tested)
## Feel free to comment out

X = [20, 23, 26, 299, np.nan, np.nan, np.nan, np.nan]
Y = [3, 4, 5, 2, 1, 2, 3, 1]
cols_test2 = pd.DataFrame([X,Y]).T
print(X,Y)
#cols_test2

[20, 23, 26, 299, nan, nan, nan, nan] [3, 4, 5, 2, 1, 2, 3, 1]


In [5]:
## define function to check NaN/null age and assign age
def impute_age_pd_loc(cols):
    age = cols[0] # assign first column to age Series
    #print(f'age: {age}') #debug
    age1 = cols.loc[:,0] # assign first column to age Series
    #print(f'age1: {age1}') #debug
    pclass = cols[1] #cols.loc[:,1] # wassign 2nd to pclass

    ## nested pd.where
    ################
    # Unsuccessful #
    ################
    #age_null = age.where((cols[0].notnull()) & (cols[1]!=1), 37) #, inplace=True)
    #age_null = age.where(~(cols[0].isnull()) & (cols[1]!=2), 39) #, inplace=True)
    '''age_null = age.where((cols[0].isnull()) & (cols[1]==1), 37, axis=0
                        ).where((cols[0].isnull()) & (cols[1]==2), 39
                               ).where((cols[0].isnull()) & (np.logical_or(cols[1] != 1, cols[1] != 2)), 24) #, inplace=True)'''
    '''age_null = age.where((age.isnull()) & (np.logical_or(pclass != 1, pclass != 2)), 24
                        ).where((age.isnull()) & (pclass==2), 39
                               ).where((age.isnull()) & (pclass==1), 37) #, inplace=True)'''
    '''age_null = age.where(~(age.isnull()), age.where((age.isnull()) & (np.logical_or(pclass != 1, pclass != 2)), 24
                                                   ).where((age.isnull()) & (pclass==2), 39)) #.where( # & ~(pclass==1)'''
    '''age_null = age.where(~(age.isnull()), age.where((age.isnull()) & (np.logical_or(pclass != 1, pclass != 2)), 24
                                                   ).where((age.isnull()) & (pclass==2), 39)) #.where( # & ~(pclass==1)'''
    ################
    
    #''' #df.loc multiple condition df.isnull
    ## df.loc[(cond), data col to change] = value to change
    cols.loc[(age.isnull()) & (pclass==1), cols.columns[0]] = 37  ##NB: age is cols[0] and pclass is cols[1]
    cols.loc[(age.isnull()) & (pclass==2), cols.columns[0]] = 39
    cols.loc[(age.isnull()) & (pclass.ne(1,2)), cols.columns[0]] = 24
    age_null = age.where(~(age.isnull()))
    ## np.isnan can be used as well, however, I'm interested in keeping it natively pandas
    ## It'll be interesting to turn df.loc to nested
    #'''
    
    print(f'df.loc age col[0]: \n{age_null}')
    #print(f'df.loc age cols: \n{cols[0]}')

In [6]:
impute_age_pd_loc(cols_test)
#impute_age_pd_loc(cols_test2)

df.loc age col[0]: 
0    20.0
1    23.0
2    26.0
3    29.0
4    37.0
5    39.0
6    24.0
7    37.0
Name: 0, dtype: float64


In [7]:
## define function to check NaN/null age and assign age
## Map() | Python in-built function
## 

def impute_age_map(cols):
    age = cols[0] # assign first column to age Series
    #print(f'age: {age}') #debug
    age1 = cols.loc[:,0] # assign first column to age Series
    #print(f'age1: {age1}') #debug
    pclass = cols.loc[:,1] # assign 2nd col to pclass
    
    #map conditional fillna
    ## map | assist: https://stackoverflow.com/a/74630735/20107918
    s =  pclass.map({1: 37, 2: 39}).fillna(24) # pclass is cols[1]
    age_null = age.fillna(s) #cols[0].fillna(s)
    
    print(f'mapped age col[0]: \n{age_null}')

In [8]:
impute_age_map(cols_test)
#impute_age_map(cols_test2)

mapped age col[0]: 
0    20.0
1    23.0
2    26.0
3    29.0
4    37.0
5    39.0
6    24.0
7    37.0
Name: 0, dtype: float64


In [9]:
## define function to check NaN/null age and assign age
## pandas.DataFrame.mask
## DataFrame.mask(cond, other=nan ...)
def impute_age_mask(cols):
    age = cols[0] # assign first column to age Series
    #print(f'age: {age}') #debug
    age1 = cols.loc[:,0] # assign first column to age Series
    #print(f'age1: {age1}') #debug
    pclass = cols.loc[:,1] # assign 2nd col to pclass
    
    #mask replace conditional fillna with iterated value
    
    '''age_null = age.mask((age.isnull()) & (pclass==1), 37, inplace=True)  ##NB: age is cols[0] and pclass is cols[1]
    age_null = age.mask((age.isnull()) & (pclass==2), 39, inplace=True)
    age_null = age.mask((age.isnull()) & (pclass.ne(1,2)), 24, inplace=True)'''
    
    age.mask((age.isnull()) & (pclass==1), 37, inplace=True)  ##NB: age is cols[0] and pclass is cols[1]
    age.mask((age.isnull()) & (pclass==2), 39, inplace=True)
    age.mask((age.isnull()) & (pclass.ne(1,2)), 24, inplace=True)
    
    #print(f'mapped age mask: \n{age_null}')
    print(f'mapped age mask: \n{age}')

In [10]:
impute_age_mask(cols_test)
#impute_age_mask(cols_test2)

mapped age mask: 
0    20.0
1    23.0
2    26.0
3    29.0
4    37.0
5    39.0
6    24.0
7    37.0
Name: 0, dtype: float64


In [ ]:
## =============== =============== ##
#        Below Caveat Emptor        #
## =============== =============== ##

In [ ]:
## define function to check NaN/null age and assign age
## pandas.DataFrame.mask
## DataFrame.mask(cond, other=nan ...)
def impute_age_mask(cols):
    age = cols[0] # assign first column to age Series
    #print(f'age: {age}') #debug
    age1 = cols.loc[:,0] # assign first column to age Series
    #print(f'age1: {age1}') #debug
    pclass = cols.loc[:,1] # assign 2nd col to pclass
    
    #mask replace conditional fillna with iterated value
    ##TODO
    #s =  pclass.map({1: 37, 2: 39}).fillna(24) # pclass is cols[1]
    #age_null = age.fillna(s) #cols[0].fillna(s)
    '''age_null = age.mask((cols[0].isnull()) & (cols[1]==1), 37,
                        age.mask((cols[0].isnull()) & (cols[1]==2), 39,
                        age.mask((cols[0].isnull()) & (np.logical_or(cols[1] != 1, cols[1] != 2)), 24))).astype(int)'''

    '''age_null = age.mask((cols[0].isnull()) & (cols[1]==1), 37).mask(
                        (cols[0].isnull()) & (cols[1]==2), 39).mask(
                        (cols[0].isnull()) & (np.logical_or(cols[1] != 1, cols[1] != 2)), 24).astype(int)'''
    
    '''#age_null = age.mask((age.isnull()) & (pclass==1), 37)  ##NB: age is cols[0] and pclass is cols[1]
    age_null = age.mask((age.isnull()) & (pclass==2), 39)
    age.mask((age.isnull()) & (pclass.ne(1,2)), 24)
    #age_null = age.where(~(age.isnull()))'''
    
    '''age_null = age.mask((age.isnull()) & (pclass==1), 37, inplace=True)  ##NB: age is cols[0] and pclass is cols[1]
    age_null = age.mask((age.isnull()) & (pclass==2), 39, inplace=True)
    age.mask((age.isnull()) & (pclass.ne(1,2)), 24, inplace=True)'''
    #age_null = age.where(~(age.isnull()))
    
    age.mask((age.isnull()) & (pclass==1), 37, inplace=True)  ##NB: age is cols[0] and pclass is cols[1]
    age.mask((age.isnull()) & (pclass==2), 39, inplace=True)
    age.mask((age.isnull()) & (pclass.ne(1,2)), 24, inplace=True)
    
    #print(f'mapped age mask: \n{age_null}')
    print(f'mapped age mask: \n{age}')

In [ ]:
## pd.where | nested pd.where

In [ ]:
#### TEST Pandas .mask documentation ####

In [ ]:
df = pd.DataFrame(np.arange(10).reshape(-1, 2), columns=['A', 'B'])
df

In [ ]:
m = df % 3 == 0
df.where(m, -df)

In [ ]:
df.where(m, -df) == np.where(m, df, -df)

In [ ]:
df.where(m, -df) == df.mask(~m, -df)
